In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
layers = keras.layers

# Code for calculating LE

In [20]:
def rnn_jac(Wxh, Whh, ht, xt, phiprime):
    """
    Compute the Jacobian of the RNN with respect to the hidden state ht
    :param Wxh: input-to-hidden weight matrix (U)
    :param Whh: hidden-to-hidden weight matrix (V)
    :param ht: current hidden state
    :param xt: current input
    :param phiprime: function handle for the derivative of the activation function
    :return: Jacobian matrix
    """

    # Compute the Jacobian of the RNN with respect to ht

    alpha=Wxh@xt + Whh@ht
    J=np.diag(phiprime(alpha))@Whh
    return J

def calc_LEs(x_batches, h0, RNNlayer, activation_function_prim=lambda x:np.heaviside(x,1), k_LE=1000):
    """
    Calculate the Lyapunov exponents of a batch of sequences using the QR method.
    :param x_batches: input sequences (batch_size, T, input_size)
    :param h0: initial hidden state (batch_size, hidden_size)
    :param RNNlayer: RNN layer object (e.g., tf.keras.layers.SimpleRNN)
    :param activation_function_prim: function handle to derivative of activation function used in the RNN layer
    :param k_LE: number of Lyapunov exponents to compute
    :return: Lyapunov exponents for each batch (batch_size, k_LE)
    """
    #get dimensions
    hidden_size = h0.shape[0]
    batch_size, T, input_size = x_batches.shape
    L = hidden_size

    #get recurrent cell
    RNNcell=RNNlayer.cell

    # Choose how many exponents to track
    k_LE = max(min(L, k_LE), 1)

    #save average Lyapunov exponent over the sequence for each batch
    lyaps_batches = np.zeros((batch_size, k_LE))
    #Loop over input sequence
    for batch in range(batch_size):
        x=x_batches[batch]
        ht=h0
        #Initialize Q
        Q = tf.eye(L)
        #keep track of average lyapunov exponents
        cum_lyaps = tf.zeros((k_LE,))

        for t in range(T):
            #Get next state ht+1 by taking a reccurent step
            xt=x[t]
            _, ht = RNNcell(xt, ht)

            #Get jacobian J
            Wxh, Whh, b = rnn_layer.get_weights()
            # Transpose to match math-style dimensions
            Wxh = Wxh.T  # Now shape (units, input_dim)
            Whh = Whh.T  # Now shape (units, units)
            J = rnn_jac(Wxh, Whh, ht, xt, activation_function_prim)

            #Get the Lyapunov exponents from qr decomposition
            Q=Q@J
            Q,R=tf.linalg.qr(Q, full_matrices=False)
            cum_lyaps += tf.math.log(tf.linalg.diag_part(R[0:k_LE, 0:k_LE]))
        lyaps_batches[batch] = cum_lyaps / T
    return lyaps_batches


# Code used to test/show implementation

Start out with defining and training a toy model

In [5]:
def define_model():
    """Define and compile a simple RNN model."""
    z0 = layers.Input(shape=[None, 1])  # time steps unspecified, 1 feature
    z = layers.SimpleRNN(32, activation="tanh")(z0)
    z = layers.Dense(32, activation='relu')(z)
    z = layers.Dense(16, activation='relu')(z)
    z = layers.Dense(1)(z)

    model = keras.models.Model(inputs=z0, outputs=z)
    model.compile(loss='mse', optimizer='adam')
    return model

def train_model(model, X, y, epochs=20, batch_size=10):
    """Train model with early stopping and LR scheduler."""
    results = model.fit(
        X, y,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        verbose=1,
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(factor=0.67, patience=3, verbose=1, min_lr=1E-5),
            keras.callbacks.EarlyStopping(patience=4, verbose=1)
        ]
    )
    return results

# Create some toy data
n_samples = 300
time_steps = 20
X = np.random.rand(n_samples, time_steps, 1)  # [batch, time, features]
Y = np.random.rand(n_samples)

# Create and train model
model = define_model()
results = train_model(model, X, Y)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.2049 - val_loss: 0.0819 - learning_rate: 0.0010
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0835 - val_loss: 0.0808 - learning_rate: 0.0010
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0829 - val_loss: 0.0810 - learning_rate: 0.0010
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0726 - val_loss: 0.0808 - learning_rate: 0.0010
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0867 - val_loss: 0.0795 - learning_rate: 0.0010
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0760 - val_loss: 0.0790 - learning_rate: 0.0010
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0813 - val_loss: 0.0800 - learning_rate: 0.0010
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0797 - val_loss: 0.0789 - learning_rate: 0.0010
Epoch 9/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0816 - val_loss: 0.0783 - learning_rate: 0.0010
Epoch 10/20
27/27 ━━━━━━━━━

Now we can calulate the LEs of the model

In [21]:
#create some batches of input data and initial hidden states
batch_size = 10   # number of sequences
T = 20            # length of each sequence
input_dim = 1     # each x is a scalar
hidden_dim = 32   # size of hidden state

X = np.random.rand(batch_size, T, input_dim)
H0 = np.random.rand(batch_size, hidden_dim)

#Get the rnn layer of the model
rnn_layer=model.layers[1]

#Define the derivative of the activation function used
tanh_prim=lambda x: 1-np.pow(np.tanh(x), 2)

#calculate the LEs
number_exponents=20
LEs=calc_LEs(X,H0, rnn_layer, tanh_prim, number_exponents)
print(LEs)

(32, 1) (32, 32) (10, 32) (1,)


InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [32,32], In[1]: [10,32] [Op:MatMul] name: 